# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice

import os

import joblib, pickle

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'mobile_price_hyperdrive'

experiment=Experiment(ws, experiment_name)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_cluster = 'hyperdrive-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2",max_nodes=4)
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

In [ ]:
input_data = pd.read_csv("https://raw.githubusercontent.com/valeriapolozun/machine_learning_azure_capstone/803e04124532b8921a0b19d70aa71abaccf74a1f/Mobile_Price_Classification.csv")


In [ ]:
input_data

In [ ]:
if "data" not in os.listdir():
    os.mkdir("./data")


input_data.to_csv("data/mobile_price_data.csv", index=False)

In [ ]:
#data_store = ws.get_default_datastore()



In [ ]:
#data_store.upload(src_dir='data/',target_path='pricing', overwrite=True, show_progress=True)
#all_data = Dataset.Tabular.from_delimited_files(path = [(data_store, 'pricing/mobile_price_traindata.csv')])

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        '--max_depth': choice(10,25,50,75)
        '--min_samples_split': choice(2,3,4,5,6)
        '--max_features': choice(5,7,10,13,16)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

env = Environment.get(workspace=ws, name="AzureML-Tutorial")

compute_target = ws.compute_targets['hyperdrive-cluster']
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=env
                      )


hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=early_termination_policy,
                                     run_config=src,
                                     max_concurrent_runs=4,
                                     max_total_runs=16,                                     
                                    )


In [ ]:
#TODO: Submit your experiment

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)

# Monitor HyperDrive runs 
# You can monitor the progress of the runs with the following Jupyter widget
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print("best run details :",best_run.get_details())
print("best run file names :",best_run.get_file_names())
print("best run metrics :",best_run.get_metrics())

In [ ]:
#Save the best model
best_run.register_model(model_name = "hyperdrive_best_run.pkl", model_path = './outputs/')

print(best_run)
best_run.download_file( name= './outputs/hyper-model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service